<h1> Analysing Colombo Sri Lanka </h1>

This project will analyse the best place to set up the next pizza joint for a pizza chain in Sri Lanka to decide on which are a pizza place could be opened

In [267]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: - 
  - anaconda::ca-certificates-2019.8.28-0, anaconda::openssl-1.1.1d-h1de35cc_2
  - anaconda::ca-certificates-2019.8.28-0, defaults::openssl-1.1.1d-h1de35cc_2
  - anaconda::openssl-1.1.1d-h1de35cc_2, defaults::ca-certificates-2019.8.28-0
  - defaults::ca-certificates-2019.8.28-0, defaults::openssl-1.1.1d-h1de35ccdone

# All requested packages already installed.

Libraries imported.


The login details are added to log in tothe foursquare api

In [268]:
CLIENT_ID = 'GPVDS1QUUR23MELHCV4MEKLFHU3WNC1CHTR5RAJYVOGHG1T5' # your Foursquare ID
CLIENT_SECRET = 'HG54VQ4BMGXUI5PYSQTR3TPIFUUE5NDWYTORNRX22GWT0APR' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GPVDS1QUUR23MELHCV4MEKLFHU3WNC1CHTR5RAJYVOGHG1T5
CLIENT_SECRET:HG54VQ4BMGXUI5PYSQTR3TPIFUUE5NDWYTORNRX22GWT0APR


Kandy is taken as the center and a radius of roughly 144000 meters is used to analayse the spred of chains in sri lanka

In [269]:
address = 'kandy'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

7.2930922 80.6350768


In [270]:
search_query = 'pizza'
radius = 144000
         
print(search_query + ' .... OK!')

pizza .... OK!


In [271]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=GPVDS1QUUR23MELHCV4MEKLFHU3WNC1CHTR5RAJYVOGHG1T5&client_secret=HG54VQ4BMGXUI5PYSQTR3TPIFUUE5NDWYTORNRX22GWT0APR&ll=7.2930922,80.6350768&v=20180604&query=pizza&radius=144000&limit=100'

In [272]:
results = requests.get(url).json()
# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()
venues = results['response']['venues']

In [273]:
dataframe.shape

(50, 18)

The list extracted from the foursquare api is displayed in a dataframe

In [274]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Pizza Hut,Pizza Place,Temple Street,7.293565,80.638521,"[{'label': 'display', 'lat': 7.293565147473436...",383,141001,LK,මහනුවර,Central Province,ශ්‍රී ලංකාව,"[Temple Street, මහනුවර 141001, ශ්‍රී ලංකාව]",NaN,NaN,4bc8146a15a7ef3bea907ada
1,Domino's Pizza,Pizza Place,"No.34, Kotugodella Veediya",7.294045,80.637032,"[{'label': 'display', 'lat': 7.294045029386195...",240,NaN,LK,මහනුවර,Central Province,ශ්‍රී ලංකාව,"[No.34, Kotugodella Veediya, මහනුවර, ශ්‍රී ලංකාව]",NaN,NaN,556586a3498ecd871e692f5c
2,Pizza Hut,Pizza Place,NaN,7.287048,80.642617,"[{'label': 'display', 'lat': 7.287048158669013...",1070,NaN,LK,NaN,NaN,ශ්‍රී ලංකාව,[ශ්‍රී ලංකාව],NaN,NaN,56a88510498ec8f94b75fd1d
3,Don's Pizza,Pizza Place,NaN,7.298423,80.636863,"[{'label': 'display', 'lat': 7.298423, 'lng': ...",625,NaN,LK,මහනුවර,Central Province,ශ්‍රී ලංකාව,"[මහනුවර, ශ්‍රී ලංකාව]",NaN,NaN,51ebeeda498e74721f7fc74e
4,Pizza Hut,Pizza Place,Old Peradeniya Road,7.280803,80.619931,"[{'label': 'display', 'lat': 7.28080260359775,...",2160,NaN,LK,මහනුවර,Central Province,ශ්‍රී ලංකාව,"[Old Peradeniya Road (Mulgampola), මහනුවර, ශ්‍...",Mulgampola,NaN,57400943498e6a948bcfbdb0
5,Vito Wood Fired Pizza,Pizza Place,NaN,7.262750,80.592830,"[{'label': 'display', 'lat': 7.2627497, 'lng':...",5759,NaN,LK,NaN,Central Province,ශ්‍රී ලංකාව,[ශ්‍රී ලංකාව],NaN,NaN,5ae48ac52aff31002c4e1e87
6,Pizza Hut,Fast Food Restaurant,337 Katugastota Road,7.319601,80.629570,"[{'label': 'display', 'lat': 7.3196006, 'lng':...",3012,20000,LK,මහනුවර,Central Province,ශ්‍රී ලංකාව,"[337 Katugastota Road, මහනුවර 20000, ශ්‍රී ලංකාව]",NaN,NaN,58caae98bbec6661edcb66bb
7,Pizza Hut,Pizza Place,Mandandawala,7.480972,80.623530,"[{'label': 'display', 'lat': 7.4809723, 'lng':...",20953,NaN,LK,Matale,Central Province,ශ්‍රී ලංකාව,"[Mandandawala (Trincomallee Street), Matale, ශ...",Trincomallee Street,NaN,5c0be3789e3b65002cf33987
8,Pizza Hut,Pizza Place,No. 06,7.162992,80.572876,"[{'label': 'display', 'lat': 7.162991969395164...",16029,NaN,LK,Gampola,Central Province,ශ්‍රී ලංකාව,"[No. 06 (Kandy Road), Gampola, ශ්‍රී ලංකාව]",Kandy Road,NaN,56bf5c2dcd10d6510acb77e8
9,Pizza Hut,Pizza Place,Colombo Road,7.473344,80.352702,"[{'label': 'display', 'lat': 7.473344499494003...",37072,NaN,LK,කුරුණෑගල,North Western Province,ශ්‍රී ලංකාව,"[Colombo Road, කුරුණෑගල, ශ්‍රී ලංකාව]",NaN,NaN,528e1fc611d2d91b810982b3


In [275]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=10) 

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

THe map shows the spread of pizza joint in Sri lanka. It clearly shows a lack of pizza places towards the south of the country.
<blockquote>

It has to be noted that this does not match up to the real world scenario as it seems there is a lack of new data on the plavces available in the south.
</blockquote>

In [288]:
# define URL
address = 'sri lanka'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

url2 = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius)

results2 = requests.get(url2).json()
# tranform venues into a dataframe
dataframe2 = json_normalize(venues)
url2

7.5554942 80.7137847


'https://api.foursquare.com/v2/venues/trending?client_id=GPVDS1QUUR23MELHCV4MEKLFHU3WNC1CHTR5RAJYVOGHG1T5&client_secret=HG54VQ4BMGXUI5PYSQTR3TPIFUUE5NDWYTORNRX22GWT0APR&ll=7.5554942,80.7137847&v=20180604&radius=144000'

In [289]:
if len(results2['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results2['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [290]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

In [291]:
if len(results['response']['venues']) == 0:
    venues_map2 = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map2 = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Ecco


        
    for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
        folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map2)
        
        # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='red',
            fill_color='red',
            fill_opacity=0.6
        ).add_to(venues_map2)
        
venues_map2

TypeError: string indices must be integers

Although there seems to be no trending data on fousquare cross refenrencing with data from the tourism board Galle seems to be a good place to start a pizza joint

http://www.sltda.lk/sites/default/files/industry-report-Q2-2018.pdf